# Mapping and classifying cells using Human Lung Cell Atlas

## Environment Setup

In [1]:
import os

import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=UserWarning)

In [2]:
import scanpy as sc
import numpy as np
import pandas as pd
import scarches as sca
import gdown
import gzip
import shutil

 captum (see https://github.com/pytorch/captum).
INFO:pytorch_lightning.utilities.seed:Global seed set to 0


In [ ]:
sc.logging.print_versions()

## Download Files - Reference model, Embeddings etc.

In [3]:
if not os.path.isdir("../data"):
    os.mkdir("../data")    

In [4]:
os.chdir('../data')

In [7]:
adata = sc.read('HLCA_emb_and_metadata.h5ad')

In [ ]:
url = 'https://zenodo.org/record/6337966/files/HLCA_emb_and_metadata.h5ad'
output = 'HLCA_emb_and_metadata.h5ad'
gdown.download(url, output, quiet=False)

In [ ]:
url = 'https://zenodo.org/record/6337966/files/HLCA_reference_model.zip'
output = 'HLCA_reference_model.zip'
gdown.download(url, output, quiet=False)
shutil.unpack_archive('HLCA_reference_model.zip')
os.remove(output)

## Download query data

In [ ]:
query_data_url = "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSM5230027&format=file&file=GSM5230027%5F04%2DP103142%2DS149%2DR01%5Fraw%5Ffeature%5Fbc%5Fmatrix%2Eh5%2Egz"
query_metadata_url = "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE171668&format=file&file=GSE171668%5Flung%5Fmetadata%2Ecsv%2Egz"

In [ ]:
# Option 1
output = 'query.h5.gz'
gdown.download(query_data_url, output, quiet=False)
with gzip.open('query.h5.gz', 'rb') as f_in:
    with open('query.h5', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
os.remove("query.h5.gz")

In [ ]:
# Option 2
output = 'query.h5.gz'
gdown.download(query_data_url, output, quiet=False)
with gzip.open('query.h5.gz', 'wrb') as gz:
    h5 = gz.read()
os.remove("query.h5.gz")

In [ ]:
output = 'query_obs.csv.gz'
gdown.download(query_metadata_url, output, quiet=False)

## Preprocessing

In [ ]:
query_data_full = sc.read_10x_h5("query.h5")
    # clean up .var.index (gene names)
query_data_full.var['gene_names'] = query_data_full.var.index
query_data_full.var.index = [idx.split("___")[-1] for idx in query_data_full.var.gene_ids]
    # clean up cell barcodes:
query_data_full.obs.index = query_data_full.obs.index.str.rstrip("-1")
    # read in metadata (to select only cells of interest and remove empty drops)
query_data_meta = pd.read_csv("query_obs.csv.gz",index_col=0)
    # subset to cells from our sample
query_data_meta = query_data_meta.loc[query_data_meta.donor == "D12_4",:].copy()
    # clean up barcodes:
query_data_meta.index = [idx.split("-")[-1] for idx in query_data_meta.index]
    # subset adata to cells in metadata:
query_data_full = query_data_full[query_data_meta.index,:].copy()
    # add dataset information:
query_data_full.obs['dataset'] = 'test_dataset_delorey_regev'

In [ ]:
# gene order for scArches model
reference_gene_order = pd.read_csv('HLCA_reference_model/var_names.csv')
# reference embedding, including cell/sample/subject metadata:
reference_embedding = sc.read_h5ad('HLCA_emb_and_metadata.h5ad')

In [ ]:
ref_model_path = 'HLCA_reference_model'
query_data = sca.models.SCANVI.prepare_query_anndata(query_data_full, ref_model_path, inplace=False)

In [ ]:
query_data.obs['scanvi_label'] = 'unlabeled'

## Perform surgery

In [ ]:
batch_key = 'dataset'
query_batch = ['test_dataset_delorey_regev']

In [ ]:
surgery_epochs = 500
early_stopping_kwargs_surgery = {
    "early_stopping_metric": "elbo",
    "save_best_state_metric": "elbo",
    "on": "full_dataset",
    "patience": 10,
    "threshold": 0.001,
    "reduce_lr_on_plateau": True,
    "lr_patience": 8,
    "lr_factor": 0.1,
}

Surgery

In [ ]:
os.chdir('/notebooks')

In [ ]:
surgery_model = sca.models.SCANVI.load_query_data(
        query_data,
        ref_model_path,
        freeze_dropout = True,
    )

In [ ]:
surgery_model._unlabeled_indices = np.arange(query_data.n_obs)
surgery_model._labeled_indices = []

In [ ]:
#TODO: Check args
surgery_model.train(
    n_epochs_semisupervised=surgery_epochs,
    train_base_model=False,
    semisupervised_trainer_kwargs=dict(
        metrics_to_monitor=["accuracy", "elbo"], 
        weight_decay=0,
        early_stopping_kwargs=early_stopping_kwargs_surgery
    ),
    frequency=1
)

In [ ]:
surgery_path = 'surgery_model'
surgery_model.save(surgery_path, overwrite=True)

### Get latent representation

In [ ]:
query_data_latent = sc.AnnData(surgery_model.get_latent_representation(query_data))
query_data_latent.obs = query_data.obs.loc[query_data.obs.index,:]

# Combine embeddings

In [ ]:
query_data_latent.obs['ref_or_query'] = "query"
reference_embedding.obs['ref_or_query'] = "ref"

In [ ]:
combined_emb = reference_embedding.concatenate(query_data_latent, index_unique=None)

# UMAP

In [ ]:
sc.pp.neighbors(combined_emb, n_neighbors=30)
sc.tl.umap(combined_emb)
sc.pl.umap(combined_emb, color="ref_or_query")

Saving combined embeddings (optional)

In [ ]:
combined_emb.write_h5ad('../data/combined_embedding.h5ad')

# Label transfer

In [ ]:
url = 'https://github.com/LungCellAtlas/mapping_data_to_the_HLCA/blob/main/supporting_files/HLCA_celltypes_ordered.csv'
output_celltypes = '../data/HLCA_celltypes_ordered.csv'
gdown.download(url, output, quiet=False)


In [ ]:
url = 'https://github.com/LungCellAtlas/mapping_data_to_the_HLCA/blob/main/supporting_files/HLCA_scarches_gene_order.csv'
output_gene_order = '../data/HLCA_scarches_gene_order.csv'
gdown.download(url, output, quiet=False)

In [ ]:
cts_ordered = pd.read_csv(output_celltypes,index_col=0)    

In [ ]:
knn_transformer = sca.utils.knn.weighted_knn_transfer(
    train_adata=reference_embedding,
    train_adata_emb="X",
    n_neighbors=50,
)

In [ ]:
labels, uncert = sca.utils.knn.weighted_knn_transfer(
    query_adata=query_data_latent,
    query_adata_emb="X", # location of our joint embedding
    label_keys="Level",
    knn_model=knn_transformer,
    ref_adata_obs = reference_embedding.obs.join(cts_ordered, on='ann_finest_level')
    )

In [ ]:
uncertainty_threshold = 0.2

In [ ]:
labels.rename(columns={f"Level_{lev}":f"Level_{lev}_transfered_label_unfiltered" for lev in range(1,6)},inplace=True)
uncert.rename(columns={f"Level_{lev}":f"Level_{lev}_transfer_uncert" for lev in range(1,6)},inplace=True)

In [ ]:
combined_emb.obs = combined_emb.obs.join(labels)
combined_emb.obs = combined_emb.obs.join(uncert)

In [ ]:
# convert to arrays instead of categoricals, and set "nan" to NaN 
for col in combined_emb.obs.columns:
    if col.endswith("_transfer_uncert"):
        combined_emb.obs[col] = list(np.array(combined_emb.obs[col]))
    elif col.endswith("_transfered_label_unfiltered"):
        filtered_colname = col.replace("_unfiltered","")
        matching_uncert_col = col.replace("transfered_label_unfiltered","transfer_uncert")
        
        # also create a filtered version, setting cells with too high 
        # uncertainty levels to "Unknown"
        combined_emb.obs[filtered_colname] = combined_emb.obs[col]
        combined_emb.obs[filtered_colname].loc[combined_emb.obs[matching_uncert_col]>uncertainty_threshold] = "Unknown"
        # convert to categorical:
        combined_emb.obs[col] = pd.Categorical(combined_emb.obs[col])
        combined_emb.obs[filtered_colname] = pd.Categorical(combined_emb.obs[filtered_colname])
        # then replace "nan" with NaN (that makes colors better in umap)
        combined_emb.obs[col].replace("nan",np.nan,inplace=True)
        combined_emb.obs[filtered_colname].replace("nan",np.nan,inplace=True)

In [ ]:
print(f"Percentage of unknown per level, with uncertainty_threshold={uncertainty_threshold}:")
for level in range(1,6):
    print(f"Level {level}: {np.round(sum(combined_emb.obs[f'Level_{level}_transfered_label'] =='Unknown')/query_data.n_obs*100,2)}%")

### UMAP Uncertainties 

In [ ]:
sc.pl.umap(
    combined_emb,
    color=[f"Level_{lev}_transfer_uncert" for lev in range(1,6)],
    ncols=5)

In [ ]:
sc.pl.umap(
    combined_emb,
    color=[f"Level_{lev}_transfer_uncert" for lev in range(1,6)],
    ncols=5)

In [ ]:
sc.pl.umap(
    combined_emb,
    color="ann_finest_level")

In [ ]:
combined_emb.write_h5ad('../data',"combined_embedding_labels.h5ad")